In [1]:
!pip install pmdarima  

     |████████████████████████████████| 1.5MB 3.4MB/s 
     |████████████████████████████████| 2.1MB 14.5MB/s 
     |████████████████████████████████| 808kB 44.7MB/s 
     |████████████████████████████████| 8.7MB 40.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21
  Found existing installation: setuptools 50.3.0
    Uninstalling setuptools-50.3.0:
      Successfully uninstalled setuptools-50.3.0
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import calendar
from statsmodels.tsa.arima_model import ARIMA
#import pyramid  
import pmdarima as pm

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [3]:
from google.colab import drive 
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [4]:
data = pd.read_csv('/content/drive/My Drive/DS4A PROJECT/processed_data/final_daily_dataset.csv')

In [5]:
data.head() 

,LCLid,day,energy_sum,Acorn,day_of_week,weekend,Holiday,month,year,season
0,MAC000131,2011-12-15,9.505,ACORN-E,Thursday,0,0,12,2011,Winter
1,MAC000131,2011-12-16,14.216,ACORN-E,Friday,0,0,12,2011,Winter
2,MAC000131,2011-12-17,9.111,ACORN-E,Saturday,1,0,12,2011,Winter
3,MAC000131,2011-12-18,10.511,ACORN-E,Sunday,1,0,12,2011,Winter
4,MAC000131,2011-12-19,15.647,ACORN-E,Monday,0,0,12,2011,Winter


In [6]:
data['holiday'] = data['weekend']+data['Holiday']

In [7]:
data = data[data['Acorn']!= 'ACORN-']
data = data[data['Acorn']!= 'ACORN-U']

In [8]:
data.day.value_counts()


2012-12-18    5491
2012-11-19    5482
2012-11-21    5482
2012-11-20    5482
2012-11-18    5481
              ... 
2011-11-27      41
2011-11-26      41
2011-11-25      32
2011-11-24      25
2011-11-23      13
Name: day, Length: 829, dtype: int64

In [9]:
data.Acorn.value_counts()


ACORN-E    1022493
ACORN-Q     533107
ACORN-F     447224
ACORN-H     271861
ACORN-L     203966
ACORN-D     188974
ACORN-G     123657
ACORN-K     106623
ACORN-A      93084
ACORN-N      91994
ACORN-C      88676
ACORN-P      72165
ACORN-M      66243
ACORN-J      64850
ACORN-O      61241
ACORN-I      29477
ACORN-B      14465
Name: Acorn, dtype: int64

In [10]:
len(data)

3480100

In [11]:
data.day = pd.to_datetime(data.day)


In [12]:
data

,LCLid,day,energy_sum,Acorn,day_of_week,weekend,Holiday,month,year,season,holiday
0,MAC000131,2011-12-15,9.505,ACORN-E,Thursday,0,0,12,2011,Winter,0
1,MAC000131,2011-12-16,14.216,ACORN-E,Friday,0,0,12,2011,Winter,0
2,MAC000131,2011-12-17,9.111,ACORN-E,Saturday,1,0,12,2011,Winter,1
3,MAC000131,2011-12-18,10.511,ACORN-E,Sunday,1,0,12,2011,Winter,1
4,MAC000131,2011-12-19,15.647,ACORN-E,Monday,0,0,12,2011,Winter,0
...,...,...,...,...,...,...,...,...,...,...,...
3510428,MAC004977,2014-02-24,5.686,ACORN-E,Monday,0,0,2,2014,Winter,0
3510429,MAC004977,2014-02-25,4.042,ACORN-E,Tuesday,0,0,2,2014,Winter,0
3510430,MAC004977,2014-02-26,5.784,ACORN-E,Wednesday,0,0,2,2014,Winter,0
3510431,MAC004977,2014-02-27,5.475,ACORN-E,Thursday,0,0,2,2014,Winter,0


In [13]:
df = data.groupby(['Acorn','day'])['energy_sum','holiday'].mean().reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [14]:
df

,Acorn,day,energy_sum,holiday
0,ACORN-A,2011-12-03,9.007000,1
1,ACORN-A,2011-12-04,20.763000,1
2,ACORN-A,2011-12-05,6.020000,0
3,ACORN-A,2011-12-06,13.322000,0
4,ACORN-A,2011-12-07,6.782000,0
...,...,...,...,...
13902,ACORN-Q,2014-02-24,7.603043,0
13903,ACORN-Q,2014-02-25,7.597203,0
13904,ACORN-Q,2014-02-26,7.561125,0
13905,ACORN-Q,2014-02-27,7.581832,0


In [15]:
df['day_of_week'] = df['day'].apply(lambda x: x.weekday()) # get the weekday index, between 0 and 6
df['day_of_week'] = df['day_of_week'].apply(lambda x: calendar.day_name[x])
df.head()

,Acorn,day,energy_sum,holiday,day_of_week
0,ACORN-A,2011-12-03,9.007,1,Saturday
1,ACORN-A,2011-12-04,20.763,1,Sunday
2,ACORN-A,2011-12-05,6.020,0,Monday
3,ACORN-A,2011-12-06,13.322,0,Tuesday
4,ACORN-A,2011-12-07,6.782,0,Wednesday


In [16]:
df['month'] = df.day.dt.month

In [17]:
df.head()

,Acorn,day,energy_sum,holiday,day_of_week,month
0,ACORN-A,2011-12-03,9.007,1,Saturday,12
1,ACORN-A,2011-12-04,20.763,1,Sunday,12
2,ACORN-A,2011-12-05,6.020,0,Monday,12
3,ACORN-A,2011-12-06,13.322,0,Tuesday,12
4,ACORN-A,2011-12-07,6.782,0,Wednesday,12


In [18]:
sp = list(np.where((df.month.values == 3) | (df.month.values == 4) | (df.month.values == 5))[0])
su = list(np.where((df.month.values == 6) | (df.month.values == 7) | (df.month.values == 8))[0])
au = list(np.where((df.month.values == 9) | (df.month.values == 10) | (df.month.values == 11))[0])
wi = list(np.where((df.month.values == 12) | (df.month.values == 1) | (df.month.values == 2))[0])

In [19]:
df['season'] = ''
df.loc[sp, 'season'] = 'Spring'
df.loc[su, 'season'] = 'Summer'
df.loc[au, 'season'] = 'Autumn'
df.loc[wi, 'season'] = 'Winter'

In [20]:
df.head()

,Acorn,day,energy_sum,holiday,day_of_week,month,season
0,ACORN-A,2011-12-03,9.007,1,Saturday,12,Winter
1,ACORN-A,2011-12-04,20.763,1,Sunday,12,Winter
2,ACORN-A,2011-12-05,6.020,0,Monday,12,Winter
3,ACORN-A,2011-12-06,13.322,0,Tuesday,12,Winter
4,ACORN-A,2011-12-07,6.782,0,Wednesday,12,Winter


In [21]:
df = df[df['day']>='2012-01-23']

In [22]:
df

,Acorn,day,energy_sum,holiday,day_of_week,month,season
51,ACORN-A,2012-01-23,22.139083,0,Monday,1,Winter
52,ACORN-A,2012-01-24,15.078583,0,Tuesday,1,Winter
53,ACORN-A,2012-01-25,16.382583,0,Wednesday,1,Winter
54,ACORN-A,2012-01-26,16.535154,0,Thursday,1,Winter
55,ACORN-A,2012-01-27,16.084000,0,Friday,1,Winter
...,...,...,...,...,...,...,...
13902,ACORN-Q,2014-02-24,7.603043,0,Monday,2,Winter
13903,ACORN-Q,2014-02-25,7.597203,0,Tuesday,2,Winter
13904,ACORN-Q,2014-02-26,7.561125,0,Wednesday,2,Winter
13905,ACORN-Q,2014-02-27,7.581832,0,Thursday,2,Winter


In [23]:
train = df[df['day']<='2013-10-31']
validation = df[df['day']>'2013-10-31']

In [24]:
len(train.day.value_counts())

648

In [25]:
len(validation.day.value_counts())

120

In [26]:
Acorn = list(df.Acorn.value_counts().index) 

In [27]:
len(Acorn)


17

In [28]:
final  = pd.DataFrame(columns = ['energy','Acorn','day']) 


In [29]:
final

,energy,Acorn,day


In [30]:
energy = []
Acorn_=[]
date = []
actual = []

In [31]:

for i in range(len(Acorn)):
  print('a')
  time1 = train[train['Acorn']==Acorn[i]]
  time = time1[['day','energy_sum']]
  #time = time.set_index('day')
  holiday = np.asarray(time1['holiday'])
  season = np.asarray(time1['season'])
  ex = np.asarray([holiday,season])

  val_time1 = validation[validation['Acorn']==Acorn[i]]
  val_time = val_time1[['day','energy_sum']]
  val_time = val_time.set_index('day')
  #print(val_time.index)
  val_holiday = np.asarray(val_time1['holiday'])
  val_season = np.asarray(val_time1['season'])
  val_ex = np.asarray([val_holiday,val_season])
  
  # fill missing dates with 0 call_volume
  r = pd.date_range(min(time.day), max(time.day))
  time=time.set_index('day').reindex(r).fillna(value=None, method='backfill', axis=None, limit=None, downcast=None)
  #print(time)
      
  #model
  arima_model = pm.auto_arima(time, 
                            start_p=1, 
                            start_q=1,
                            exogeneous = ex)
  future_forecast = arima_model.predict(n_periods=120, exogenous= val_ex)
  #print(future_forecast)
  #future_forecast = pd.DataFrame(future_forecast,columns=['energy'])
  #future_forecast['Acorn'] = Acorn[i]  
  #future_forecast['day'] = val_time1.index
  #print(future_forecast)
  a = [Acorn[i]]*120 
  actual.extend(list(val_time['energy_sum'])) 
  Acorn_.extend(a)
  energy.extend(future_forecast)
  date.extend(val_time.index)
  # final.append(future_forecast) 
  # print(final)  

a


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


a


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: 

a


...

a


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmo

a
a


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.p

a


In [32]:
final = pd.DataFrame(list(zip(Acorn_, energy,actual, date)), 
               columns =['Acorn', 'energy','Actual','date']) 

In [33]:
final = final.loc[final['date']!='2014-02-28'] 

In [34]:
final['error'] = (final['Actual'] - final['energy'])/final['Actual'] 

In [35]:
f1 = final.groupby('Acorn')[['Actual','energy']].sum().reset_index()

In [42]:
f1['error'] = (f1['Actual']-f1['energy'])/f1['Actual']  
f1[['Acorn','error']].iloc[:9]

,Acorn,error
0,ACORN-A,0.094074
1,ACORN-B,0.147310
2,ACORN-C,0.115035
3,ACORN-D,0.122267
4,ACORN-E,0.159651
5,ACORN-F,0.107309
6,ACORN-G,0.166908
7,ACORN-H,0.104386
8,ACORN-I,0.067316


In [43]:
f1[['Acorn','error']].iloc[9:]

,Acorn,error
9,ACORN-J,0.142153
10,ACORN-K,0.113854
11,ACORN-L,0.094621
12,ACORN-M,0.105504
13,ACORN-N,0.114482
14,ACORN-O,0.157981
15,ACORN-P,0.134235
16,ACORN-Q,0.094344
